In [25]:
import pandas as pd
import requests
import os
from square.client import Client
import config
import psycopg2
import json
from square.http.auth.o_auth_2 import BearerAuthCredentials
from sqlalchemy import create_engine 


client = Client(
    bearer_auth_credentials=BearerAuthCredentials(
        access_token=config.square_access_token
    ),
    environment=config.square_environment,
)


In [18]:
result = client.payments.list_payments(
    begin_time="2023-01-01T12:00:00Z",
    sort_order="desc",
    location_id="LMFGQ336T5DF1",
)

if result.is_success():
    data = result.body.get('payments',[])
    df = pd.json_normalize(data)
else:
    print(f"Unable to fetch data: {result.status_code}")

API returns JSON arrays, need to figure out how to read these

In [19]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 57 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    100 non-null    object 
 1   created_at                                            100 non-null    object 
 2   updated_at                                            100 non-null    object 
 3   status                                                100 non-null    object 
 4   source_type                                           100 non-null    object 
 5   location_id                                           100 non-null    object 
 6   order_id                                              100 non-null    object 
 7   processing_fee                                        44 non-null     object 
 8   customer_id                                           3 non-n

In [26]:

conn_params_dic = {
    "host": "localhost:5432",
    "database": "airflow",
    "user": "airflow",
    "password": "airflow",
}
conn_string = f"postgresql+psycopg2://{conn_params_dic['user']}:{conn_params_dic['password']}@{conn_params_dic['host']}/{conn_params_dic['database']}"
engine = create_engine(conn_string)

df.to_sql('listpayments', con=engine, if_exists='replace', index=False)

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO listpayments (id, created_at, updated_at, status, source_type, location_id, order_id, processing_fee, customer_id, employee_id, receipt_number, receipt_url, team_member_id, version_token, "amount_money.amount", "amount_money.currency", "card_details.status", "card_details.card.card_brand", "card_details.card.last_4", "card_details.card.exp_month", "card_details.card.exp_year", "card_details.card.fingerprint", "card_details.card.card_type", "card_details.card.prepaid_type", "card_details.card.bin", "card_details.entry_method", "card_details.cvv_status", "card_details.avs_status", "card_details.auth_result_code", "card_details.statement_description", "card_details.card_payment_timeline.authorized_at", "card_details.card_payment_timeline.captured_at", "total_money.amount", "total_money.currency", "approved_money.amount", "approved_money.currency", "application_details.square_product", "application_deta